In [1]:
from dgd.utils.utils5 import load_graph_pickle, energy_score, check_implicit_OR_existence_v3

In [5]:
# --- Cell 1: imports & helpers ---
from typing import Callable, Optional
import networkx as nx

# Your project utilities
from dgd.utils.utils5 import load_graph_pickle, energy_score, check_implicit_OR_existence_v3

def _to_graph(x):
    if isinstance(x, (nx.Graph, nx.DiGraph)):
        return x
    if isinstance(x, dict):
        return nx.node_link_graph(x)
    return nx.node_link_graph(x)  # will raise if not valid

def resolve_current_solution_getter(env) -> Callable[[], object]:
    """
    Returns a zero-arg getter that retrieves the current solution, whether
    it's stored as a value or exposed as a callable.
    """
    # Try wrapper attr first
    try:
        attr = env.get_wrapper_attr("current_solution")
        if callable(attr):
            return lambda: attr()
        if attr is not None:
            return lambda a=attr: a
    except Exception:
        pass

    # Try unwrapped as backup (still "current_solution" only)
    try:
        attr = getattr(env.unwrapped, "current_solution", None)
        if callable(attr):
            return lambda: attr()
        if attr is not None:
            return lambda a=attr: a
    except Exception:
        pass

    # Could not find it
    return lambda: None

def current_solution_energy(env) -> Optional[float]:
    getter = resolve_current_solution_getter(env)
    cur = getter()
    if cur is None:
        return None
    G = _to_graph(cur)
    e, _ = energy_score(G, check_implicit_OR_existence_v3)
    return float(e)

def debug_current_solution(env):
    """
    Prints whether current_solution is callable or a value,
    shows the type of the resolved object, and computes its energy.
    """
    # Inspect wrapper attribute directly (for the "callable?" question)
    attr_kind = "missing"
    try:
        attr = env.get_wrapper_attr("current_solution")
        if attr is None:
            attr_kind = "None"
        elif callable(attr):
            attr_kind = "callable"
        else:
            attr_kind = f"value ({type(attr).__name__})"
    except Exception as e:
        attr = None
        attr_kind = f"unavailable via get_wrapper_attr ({type(e).__name__})"

    # Also check unwrapped for completeness
    try:
        raw = getattr(env.unwrapped, "current_solution", None)
        raw_kind = "None" if raw is None else ("callable" if callable(raw) else f"value ({type(raw).__name__})")
    except Exception as e:
        raw_kind = f"unavailable ({type(e).__name__})"

    # Resolve through the unified getter and compute energy
    try:
        getter = resolve_current_solution_getter(env)
        resolved = getter()
        resolved_type = "None" if resolved is None else type(resolved).__name__
    except Exception as e:
        resolved = None
        resolved_type = f"error resolving ({type(e).__name__})"

    print(f"[current_solution via wrapper] -> {attr_kind}")
    print(f"[current_solution via unwrapped] -> {raw_kind}")
    print(f"[resolved getter type] -> {resolved_type}")

    if resolved is not None:
        try:
            e = current_solution_energy(env)
            print(f"[energy from current_solution] -> {e}")
        except Exception as e:
            print(f"[energy computation failed] -> {e}")
    else:
        print("[no current_solution to compute energy]")



In [3]:
# --- Cell 2 (optional): build or obtain an env ---
# If you already have 'env', skip this cell.

# Example: using your SA/backtracking env factory from your script (if it's importable)
# Otherwise, construct your env the way you normally do in your project.

from dgd.environments.drl3env_loader6_with_trajectories_and_backtracking import DRL3env as DRL3envBacktrack
from multiprocessing import Manager

# Set your ID and seed path
ID = "0x0239"  
seed_path = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/dgd/data/NIGs_4_inputs/{ID}_NIG_unoptimized.pkl"
G0 = load_graph_pickle(seed_path)

# Minimal objects the env expects (mirroring your runner’s pattern)
mgr = Manager()
lock = mgr.Lock()
best = mgr.Value('d', float('inf'))

# Start the registry with just this one seed
from dgd.utils.utils5 import check_implicit_OR_existence_v3  # already imported above
from dgd.environments.drl3env_loader6_with_trajectories import _compute_truth_key, _apply_implicit_or
from pathlib import Path

def init_singleton_registry(mgr, G, lock):
    reg = mgr.dict()
    from dgd.utils.utils5 import energy_score
    canon = _apply_implicit_or(G.copy())
    e, _ = energy_score(G, check_implicit_OR_existence_v3)
    key = _compute_truth_key(G)
    reg[key] = [(canon, G.copy(), float(e))]
    return reg

registry = init_singleton_registry(mgr, G0, lock)
existing_keys = {_compute_truth_key(G0)}

env = DRL3envBacktrack(
    max_nodes=100,
    graphs=[G0],
    shared_registry=registry,
    registry_lock=lock,
    store_every_new_graph=True,
    sampling_from_shared_registry=False,
    registry_read_only=False,
    max_steps=50,
    enable_full_graph_replacement=True,
    show_plots=False,
    log_info=False,
    strict_iso_check=False,
    initial_state_sampling_factor=0.0,
    existing_keys=existing_keys,
    best_energy_across_workers=best,
)

# If your project wraps with ActionMasker, that’s also fine:
from sb3_contrib.common.wrappers import ActionMasker
def mask_fn(env_): return env_.action_masks()
env = ActionMasker(env, mask_fn)

# Reset once to initialize internal state/solution
_ = env.reset(seed=123)


Done loading action motifs. There are 15928 unique motifs.


In [6]:
# --- Cell 3: run the check ---
debug_current_solution(env)



[current_solution via wrapper] -> value (DiGraph)
[current_solution via unwrapped] -> value (DiGraph)
[resolved getter type] -> DiGraph
[energy from current_solution] -> 36.0
